# To Start Credentials

In [1]:
# THIS CELL READS RESOURCE GROUPS
# https://docs.microsoft.com/en-us/azure/developer/python/sdk/examples/azure-sdk-example-resource-group
# https://docs.microsoft.com/en-us/azure/developer/python/sdk/examples/azure-sdk-example-list-resource-groups
# Import the needed credential and management objects from the libraries.
from azure.identity import AzureCliCredential, ClientSecretCredential, DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
import os

# Acquire a credential object using CLI-based authentication.
credential = AzureCliCredential()

# Acquire a credential object using ClientSecretCredential
AZURE_TENANT_ID=os.environ["AZURE_TENANT_ID"]
AZURE_CLIENT_ID=os.environ["AZURE_CLIENT_ID"]
AZURE_CLIENT_SECRET=os.environ["AZURE_CLIENT_SECRET"]
# AZURE_SUBSCRIPTION_ID=os.environ("AZURE_SUBSCRIPTION_ID")


# Get resource group list

In [4]:
# credential = ClientSecretCredential(tenant_id=AZURE_TENANT_ID, client_id=AZURE_CLIENT_ID, client_secret=AZURE_CLIENT_SECRET)
# credential =  DefaultAzureCredential()

# Retrieve subscription ID from environment variable.
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)

# Retrieve the list of resource groups
group_list = resource_client.resource_groups.list()

# Show the groups in formatted output
column_width = 40

print("Resource Group".ljust(column_width) + "Location")
print("-" * (column_width * 2))

for group in list(group_list):
    print(f"{group.name:<{column_width}}{group.location}")

Resource Group                          Location
--------------------------------------------------------------------------------
PythonAzureExample-rg                   westeurope


## TO MAKE A RESOURCE GROUP

In [5]:
# Import the needed credential and management objects from the libraries.
from azure.mgmt.resource import ResourceManagementClient
from azure.identity import AzureCliCredential
import os

# Acquire a credential object using CLI-based authentication.
credential = AzureCliCredential()

# Retrieve subscription ID from environment variable.
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)


# https://docs.microsoft.com/en-us/dotnet/api/microsoft.azure.documents.locationnames.centralus?view=azure-dotnet
# CHECK HERE THE LOCATIONS
# Provision the resource group.
rg_result = resource_client.resource_groups.create_or_update(
    "PythonAzureExample-rg",
    {
        "location": "westeurope"
    }
)

# Within the ResourceManagementClient is an object named resource_groups,
# which is of class ResourceGroupsOperations, which contains methods like
# create_or_update.
#
# The second parameter to create_or_update here is technically a ResourceGroup
# object. You can create the object directly using ResourceGroup(location=LOCATION)
# or you can express the object as inline JSON as shown here. For details,
# see Inline JSON pattern for object arguments at
# https://docs.microsoft.com/azure/developer/python/azure-sdk-overview#inline-json-pattern-for-object-arguments.

print(f"Provisioned resource group {rg_result.name} in the {rg_result.location} region")

# The return value is another ResourceGroup object with all the details of the
# new group. In this case the call is synchronous: the resource group has been
# provisioned by the time the call returns.

# To update the resource group, repeat the call with different properties, such
# as tags:
group_config =    {
        "location": "westeurope",
        "tags": { "environment":"test", "department":"tech" }
    }
rg_result = resource_client.resource_groups.create_or_update(
    "PythonAzureExample-rg",group_config
)

print(f"Updated resource group {rg_result.name} with tags")

# Optional lines to delete the resource group. begin_delete is asynchronous.
# poller = resource_client.resource_groups.begin_delete(rg_result.name)
# result = poller.result()

Provisioned resource group PythonAzureExample-rg in the westeurope region
Updated resource group PythonAzureExample-rg with tags


# Delete a resource group

In [13]:
delete_async_operation = resource_client.resource_groups.begin_delete('PythonAzureExample-rg')
delete_async_operation.wait()

CONTAINER INSTANCE MANAGEMENT CLIENT (HOPE THIS WORKS)

# Get a Resource Group

In [7]:
import resource
from azure.mgmt.containerinstance import ContainerInstanceManagementClient
from azure.mgmt.containerinstance.models import (ContainerGroup,
                                                 Container,
                                                 ContainerGroupNetworkProtocol,
                                                 ContainerGroupRestartPolicy,
                                                 ContainerPort,
                                                 EnvironmentVariable,
                                                 IpAddress,
                                                 Port,
                                                 ResourceRequests,
                                                 ResourceRequirements,
                                                 OperatingSystemTypes)


container_group_name = 'PythonAzureExample-rg'
resource_group = resource_client.resource_groups.get(container_group_name)
print(resource_group.name)
print(resource_group.location)


PythonAzureExample-rg
westeurope


# Create single Instance

In [8]:

# container_image_name = "mcr.microsoft.com/azuredocs/aci-helloworld"
container_image_name = "stablebaselines/rl-baselines3-zoo-cpu"

start_command_line = "python train.py --algo ppo --env CartPole-v1"


# Obtain the container object for instances.
instance_client = ContainerInstanceManagementClient(credential, subscription_id)
# Configure the container
container_resource_requests = ResourceRequests(memory_in_gb=1, cpu=1.0)
container_resource_requirements = ResourceRequirements(
    requests=container_resource_requests)
container_name = "container-1"
container = Container(name=container_name,
                        image=container_image_name,
                        resources=container_resource_requirements,
                        command=start_command_line.split(),
                        ports=[ContainerPort(port=80)])
# Configure the container group
ports = [Port(protocol=ContainerGroupNetworkProtocol.tcp, port=80)]
group_ip_address = IpAddress(ports=ports,
                                dns_name_label=container_group_name,
                                type="Public")
group = ContainerGroup(location=resource_group.location,
                        containers=[container],
                        os_type=OperatingSystemTypes.linux,
                        ip_address=group_ip_address)

# # Create the container group
instance_client.container_groups.begin_create_or_update(resource_group.name,
                                                container_name,
                                                group)


# Get Container group of a single instance

In [8]:

# Get the created container group
container_group = instance_client.container_groups.get(resource_group.name,
                                                    container_name)

# print("Once DNS has propagated, container group '{0}' will be reachable at"
#         " http://{1}".format(container_group_name,
#                             container_group.ip_address.fqdn))


# Delete Single Container

In [9]:
instance_client.container_groups.begin_delete(resource_group.name,
                                                container_name)

# Create multiple container instances on a resource group

In [16]:

container_image_name = "mcr.microsoft.com/azuredocs/aci-helloworld"
# Obtain the container object for instances.
instance_client = ContainerInstanceManagementClient(credential, subscription_id)
# Configure the container
container_resource_requests = ResourceRequests(memory_in_gb=1, cpu=1.0)
container_resource_requirements = ResourceRequirements(
    requests=container_resource_requests)
container_group_name = "containergg"
container_1 = Container(name=container_group_name+'-1',
                        image=container_image_name,
                        resources=container_resource_requirements,
                        ports=[ContainerPort(port=80)])
container_2 = Container(name=container_group_name+'-2',
                        image=container_image_name,
                        resources=container_resource_requirements)
# Configure the container group
ports = [Port(protocol=ContainerGroupNetworkProtocol.tcp, port=80)]
group_ip_address = IpAddress(ports=ports,
                                dns_name_label=container_group_name,
                                type="Public")
group = ContainerGroup(location=resource_group.location,
                        containers=[container_1, container_2],
                        os_type=OperatingSystemTypes.linux,
                        ip_address=group_ip_address)

# # Create the container group
instance_client.container_groups.begin_create_or_update(resource_group.name,
                                                container_group_name,
                                                group)


# Delete Container Group

In [18]:
instance_client.container_groups.begin_delete(resource_group.name,
                                                container_group_name)

# Create a task based container